In [1]:
import pandas as pd
import numpy as np
from custom_plots import plot_categorical_comparison, plot_correlation_matrix, plot_time_series
from sklearn.preprocessing import MinMaxScaler

In [2]:
COLOR_DICT = {
                'cdn': '#B32E29',
                'us': '#3279B3',
                'uk': '#DE7012',
                'int': '#FACD14',
                'alberta':'#587AFF',
                'british columbia':'#37BD54',
                'manitoba': '#FFD140',
                'nova scotia' : '#FF9A30',
                'ontario' : '#BD2F24',
                'quebec' : '#60C3FF',
                'new brunswick' : '#9552FF'
             }

In [3]:
df = pd.read_csv('data/clean_data.csv')

In [4]:
df.columns

Index(['Unnamed: 0', 'week_of', 'station', 'station_city', 'station_province',
       'station_latitude', 'station_longitude', 'chart_position',
       'artist_name(s)', 'artist_country', 'artist_home_city',
       'artist_home_latitude', 'artist_home_longitude',
       'km_distance_(home_station)', 'album_name', 'label_name', 'label_type',
       'language_of_music', 'visible_ethnic_minority',
       'census_race_classification', 'artist_gender', 'm_music', 'a_artist',
       'p_performance', 'l_lyrics', 'artist_is_group', 'canadian_content',
       'city_population'],
      dtype='object')

In [26]:
plot_categorical_comparison(
                                        categories = df['census_race_classification'], 
                                        compare_by = df['artist_country'], 
                                        comparison_filter=['us','cdn','uk','int'], 
                                        color_dict=COLOR_DICT
                                        )

In [6]:
plot_categorical_comparison(
                                        categories = df['artist_gender'], 
                                        compare_by = df['artist_country'], 
                                        comparison_filter=['us','cdn','uk','int'], 
                                        color_dict=COLOR_DICT )

In [7]:
named_df = df
named_df['visible_ethnic_minority'].replace(['yes','no'],['bipoc','white'],inplace=True)
plot_categorical_comparison(
                                        categories = named_df['visible_ethnic_minority'], 
                                        compare_by = named_df['station_province'], 
                                        category_filter=['bipoc','white'], 
                                        color_dict=COLOR_DICT )


In [8]:
plot_categorical_comparison(
                                        categories = df['canadian_content'], 
                                        compare_by = df['station_province'], 
                                        # category_filter=['yes','no'], 
                                        color_dict=COLOR_DICT )

In [9]:
station_df = pd.read_csv('data/clean_data_stations.csv')
station_df[station_df['station_province'] == 'nova scotia']

,Unnamed: 0,station,station_city,city_population,station_province,station_latitude,station_longitude,total_plays,total_artists,artists_to_plays_ratio,artists_to_population_ratio,bipoc_artists,canadian_artists,male_artists,english_plays
6,6,ckdu,halifax,403130,nova scotia,44.65,-63.6,780,169,0.216667,0.000419,0.520513,0.75641,0.916667,0.979487


In [12]:
station_df = pd.read_csv('data/clean_data_stations.csv', index_col=0)
station_df.head(1)

,station,station_city,city_population,station_province,station_latitude,station_longitude,total_plays,total_artists,artists_to_plays_ratio,artists_to_population_ratio,bipoc_artists,canadian_artists,male_artists,english_plays
0,cjsr,edmonton,932550,alberta,53.55,-113.5,1920,425,0.221354,0.000456,0.651042,0.432292,0.851562,0.986458


In [13]:
station_correlations = station_df[[
                                'total_plays',
                                'total_artists',
                                'bipoc_artists',
                                'canadian_artists',
                                'male_artists',
                                'english_plays',
                                'city_population'
                            ]]
plot_correlation_matrix(station_correlations, title='Correlation Map of Station Data')

In [16]:
artist_df = pd.read_csv('data/clean_data_artists.csv',index_col=0)

In [17]:
artist_df.columns

Index(['artist_name(s)', 'artist_country', 'artist_home_city',
       'visible_ethnic_minority', 'census_race_classification',
       'artist_gender', 'canadian_content', 'artist_is_group', 'total_plays',
       'alberta', 'british columbia', 'manitoba', 'new brunswick',
       'nova scotia', 'ontario', 'quebec', 'artist_country_cdn',
       'artist_country_int', 'artist_country_uk', 'artist_country_unknown',
       'artist_country_us', 'visible_ethnic_minority_no',
       'visible_ethnic_minority_unknown', 'visible_ethnic_minority_yes',
       'artist_gender_female', 'artist_gender_male', 'artist_gender_mixed',
       'artist_gender_unknown'],
      dtype='object')

In [18]:
artist_correlations = artist_df[[
                                'total_plays',
                                'canadian_content',
                                'visible_ethnic_minority_yes',
                                'artist_gender_male',
                                'artist_is_group'
                                ]]
plot_correlation_matrix(artist_correlations._get_numeric_data(),title='Correlation Map of Artist Data')

In [19]:
time_table = pd.read_csv('data/time_table.csv', index_col=0)

In [20]:
time_table = time_table.sort_values('week_of')

In [21]:
x = time_table['week_of']
y = [x.split('_')[0] for x in time_table.columns if 'station' in x]
z = time_table[[x for x in time_table.columns if 'station' in x]].T

plot_time_series(x,y,z, title='Total Plays by Station')

In [22]:
x = time_table['week_of']
y = [x.split('_')[0] for x in time_table.columns if 'province' in x]
z = time_table[[x for x in time_table.columns if 'province' in x]].T
plot_time_series(x,y,z, 'Total Plays by Province')

In [23]:
x = time_table['week_of']
y = ['bipoc_plays','canadian_content_plays']
z = time_table[y].T
plot_time_series(x,y,z, title='BIPOC and Canadian Content Plays')

In [24]:
import plotly.graph_objects as go
from scipy import signal
fig = go.Figure()

fig.add_trace(go.Scatter(
                        x=time_table['week_of'],
                        y=time_table['bipoc_plays'],
                        fill='toself',
                        fillcolor='rgba(39, 107, 54, 0.25)',
                        line_color='rgba(39, 107, 54, 0.5)',
                        showlegend=True,
                        name='BIPOC',
))
fig.add_trace(go.Scatter(
                        x=time_table['week_of'],
                        y=time_table['canadian_content_plays'],
                        fill='toself',
                        fillcolor='rgba(132, 20, 92, 0.25)',
                        line_color='rgba(132, 20, 92, 0.5)',
                        showlegend=True,
                        name='Canadian Content',
))

fig.update_layout(title='BIPOC and Canadian Content plays over time',
                  width=800)
fig.show()

In [25]:
x = time_table['week_of']
y = [int(x.split('_')[0]) for x in time_table.columns if 'chart_position' in x]
y.sort(reverse=True)
y = [str(x) for x in y]
z = time_table[[x for x in time_table.columns if 'chart_position' in x]].T

plot_time_series(x,y,z, title='Chart Positions of Songs Played')